In [1]:
from selenium import webdriver

In [2]:
from bs4 import BeautifulSoup
import hashlib,time,pymysql

In [3]:
chrome = webdriver.Chrome()

In [27]:
#chrome.get("https://www.toutiao.com/c/user/4358506407/#mid=4358506407") # 河北卫视中华好诗词　(已完成)
# target_url = "https://www.toutiao.com/c/user/61825516347/#mid=1569872688391170" 
# target_url = "https://www.toutiao.com/c/user/70173980542/#mid=1588494198353934"
# target_url = "https://www.toutiao.com/c/user/63312853690/#mid=1572722391216142"
target_url = "https://www.toutiao.com/c/user/6496691963/#mid=6741216600"
chrome.get(target_url)

In [28]:
chrome.page_source

'<html><head>\n    <meta http-equiv="x-dns-prefetch-control" content="on">\n    <link rel="dns-prefetch" href="//s3.pstatp.com/">\n    <link rel="dns-prefetch" href="//s3a.pstatp.com/">\n    <link rel="dns-prefetch" href="//s3b.pstatp.com">\n    <link rel="dns-prefetch" href="//p1.pstatp.com/">\n    <link rel="dns-prefetch" href="//p3.pstatp.com/">\n    <meta charset="utf-8">\n    <meta http-equiv="Content-Security-Policy" content="upgrade-insecure-requests">\n    <meta http-equiv="Content-Type" content="text/html; charset=utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n    <meta name="renderer" content="webkit">\n    \n    \n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, minimal-ui">\n    \n    <meta name="baidu-site-verification" content="T239HZAbh7">\n    \n    <meta name="360-site-verification" content="b96e1758dfc9156a410a4fb9520c5956">\n    <meta name="360_ssp_verify" c

In [29]:
soup = BeautifulSoup(chrome.page_source)

In [30]:
data = soup.findAll("li",{"ga_event":"feed_item_click"})

In [31]:
data[0]

<li ad_id="" ad_track="" class="item" ga_event="feed_item_click"> <span id="ad_extra" style="display:none;"></span> <div class="item-inner y-box"> <div class="normal rbox"> <div class="rbox-inner"> <div class="title-box" ga_event="article_title_click"> <a class="link title" href="/item/6724523817185575431/" target="_blank">真正的常胜将军贾复：功冠天下，却为何主动削甲兵而弃兵权？</a> </div> <div class="y-box footer"> <div class="y-left"> <span class="lbtn ltop">置顶</span> <a class="lbtn" ga_event="article_read_count" href="/item/6724523817185575431/" target="_blank">1.6万阅读 ⋅</a> <a class="lbtn comment" ga_event="article_comment_click" href="/item/6724523817185575431//#comment_area" target="_blank"> 0评论 </a> <span class="lbtn">⋅ 2019-08-13 13:46</span> </div> <div class="y-right"> </div> </div> </div> </div> <div class="lbox" ga_event="article_img_click"> <a class="img-wrap" href="/item/6724523817185575431/" target="_blank"> <img alt="" src="//p1.pstatp.com/list/190x124/pgc-image/134577490587455abef3b7994cc422e2"/> 

In [25]:
len(data)

109

In [19]:
data[0].a['href']

'/item/6684105728782762499/'

In [9]:
len(data)

55

In [18]:
data[6].find("i")

In [20]:
ftype_count = 0

In [11]:
def create_connection():
    connection = pymysql.connect(host='www.voiceofmatrix.cn',port=3306,user='root',password='DiscuzMysql819.',database='discuz')
    return connection

def create_conn():
    conn = create_connection()
    return conn

def create_cursor(conn):
    return conn.cursor()

def execute_cmd(cursor,cmd):
    cursor.execute(cmd)

def conn_close(conn):
    conn.commit()
    conn.close()
    
def construct_sql(spiderid,url,subject,sha1,group_id,createtime):
    sql = '''INSERT IGNORE discuz_addon_collect_toutiao_articlelist(spiderid,url,subject,group_id,sha1,posttype,creattime,updatetime)
            VALUES({0},'{1}','{2}',{3},'{4}',{5},{6},{7})'''.format(spiderid,url,subject,group_id,sha1,posttype,createtime,createtime)
    
    return sql

In [32]:
# 1:河北卫视中华好诗词
# 2:今日头条庆余
# 3:今日头条诗词世界
# 4: 今日头条古今通史
# 5:读历见闻
# 6: 历史评说 
spider_id = 6
url_prefix = "http://www.toutiao.com/group/"

conn = create_connection()
cursor = create_cursor(conn)


for item in data:
    if(item.find("i")):
        continue
    else:
#         print(item.a.text, url_prefix + item.a['href'].split("/")[2] + "/")
        subject = item.a.text
        group_id = item.a['href'].split("/")[2]
        url = url_prefix + group_id + "/"
        sha1 = hashlib.sha1(bytearray(url,'utf-8')).hexdigest()
        posttype=0
        status=0
        createtime = int(time.time())
        updatetime = createtime
        
        sql_cmd = construct_sql(spider_id,url,subject,sha1,group_id,createtime)
        
        execute_cmd(cursor,sql_cmd)
conn_close(conn)

/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'message' doesn't have a default value")
  result = self._query(query)
/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1364, "Field 'errlog' doesn't have a default value")
  result = self._query(query)
/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '4ed1b460de185994fc6b23d4930c6296ec164df2' for key 'sha1'")
  result = self._query(query)
/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry 'ca9986a2ed4a1778a5c9890cb85479bc1305bf2a' for key 'sha1'")
  result = self._query(query)
/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry '2ca91ca02d912d4d9d3f2f5b1c5f0504be7c1329' for key 'sha1'")
  result = self._query(query)
/usr/local/lib/python3.6/dist-packages/pymysql/cursors.py:170: Warning: (1062, "Duplicate entry 'e82a7a4ab5238ee4afb253af2360a2da

In [81]:
len(data)

1152

In [44]:
if(a):
    print(1)
else:
    print(2)

2
